In [811]:
#%pip install gamspy

In [812]:
#! gamspy install license 204a1a5e-5e64-4583-a281-508165b2ee7e

In [813]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Sense, Options
import pandas as pd

# Initiated Pandas dataframes

In [814]:
products = ["A", "B", "C"]
crude_oil = ["co1", "co2"]
week = ["1", "2", "3"]

A_price = 1000 # SEK per unit
B_price = 740 # SEK per unit

CrdOil1_price = 500 # SEK per unit
CrdOil1_purchase_limit = 300 # units
CrdOil1_std = 20 # units
CrdOil1_mean = 300 # units
CrdOil1_extra = 700 # SEK per unit

CrdOil2_price = 600 # SEK per unit
CrdOil2_purchase_limit = 300 # units

CrdConv_cost = 100 # SEK per unit
CrdConv_limit = 500 # units

ReConv_cost = 80 # SEK per unit
ReConv_limit = 300 # units

Storage_cost = 20 # SEK per unit


_Oil2Prod = pd.DataFrame({
    "Product": products,
    "co1":  [0.5, 0.3,  0.2],
    "co2":  [0.7, 0.2,  0.1],
})

_Prod2Prod = pd.DataFrame({
    "Product": products,
    "A":        [0.0, 0.0,  0.0],
    "B":        [0.9, 0.1,  0.0],
    "C":        [0.7, 0.3,  0.0],
})

_Sell_limit = pd.DataFrame({
    "Product": products,
    "1":        [250, 250,  250],
    "2":        [30,  130,  130],
    "3":        [0,   0,    0  ],
})

_Product_criteria = pd.DataFrame({
    "Product":    products,
    "sell_price": [A_price, B_price, 0],
})

_Oil_criteria = pd.DataFrame({
    "Oil": crude_oil,
    "purchase_price": [CrdOil1_price, CrdOil2_price],
    "purchase_limit": [CrdOil1_purchase_limit, CrdOil2_purchase_limit],
})
# Gamspy is still not really good so we need to vectorize the pandas frames.
# Vectorized (melted) version
_Oil2Prod =  _Oil2Prod.melt(id_vars="Product", var_name="Oil", value_name="rate")
_Prod2Prod =  _Prod2Prod.melt(id_vars="Product", var_name="PP", value_name="rate")
_Sell_limit = _Sell_limit.melt(id_vars="Product", var_name="Week", value_name="limit")

# Initiate

In [815]:
m = Container()

In [816]:
t = Set(m,
        name=       "time",
      	description="time in weeks",
        records=week,
        )

o = Set(m,
        name=       "crude_oil",
        description="the different types of crude oil we can buy",
        records=crude_oil,
        )

p = Set(m,
        name=       "products",
        description="the different types of end products we can sell",
        records=products,
        )

pp = Alias(m, "pp", p)

# Constants

In [817]:
Purchase_Price = Parameter(m, name="oilprice", domain=[o], description="Oil price", records=_Oil_criteria[["Oil", "purchase_price"]])
Purchase_Limit = Parameter(m, name="purchaselimit", domain=[o], description="Oil purchase limit", records=_Oil_criteria[["Oil", "purchase_limit"]])

Sell_Price = Parameter(m, name="sellprice", domain=[p], description="Products sell price", records=_Product_criteria[["Product", "sell_price"]])
Sell_limit = Parameter(m, name="selllimit", domain=[p,t], description="Products sell limit", records=_Sell_limit[["Product", "Week", "limit"]])

Crude_Conv_ratio = Parameter(m, name="convratio", domain=[o,p], description="Crude oil conversion ratio", records=_Oil2Prod[["Oil", "Product", "rate"]])
Re_Conv_ratio = Parameter(m, name="reconvratio", domain=[p,pp], description="Reconversion ratio", records=_Prod2Prod[["PP", "Product", "rate"]])


# Variable (The thing we are after)

In [818]:
Oil2Buy = Variable(m, name="oil_to_buy", type="positive", domain=[o,t], description="self_explanatory")
ConvProd = Variable(m, name="converted_products", type="positive", domain=[p,t], description="self_explanatory")
Conv2Tot = Variable(m, name="converted_to_tot", type="positive", domain=[p,t], description="the products that will be at the end from conv")
ReConv2Tot = Variable(m, name="reconverted_to_tot", type="positive", domain=[p,t], description="the products that will be at the end from reconv")
Tot2Save = Variable(m, name="total_to_save", type="positive", domain=[p,t], description="self_explanatory")

# Equations

In [819]:
# Purchase parity equation
Oil2Buy.up[o,t] = Purchase_Limit[o]

# Oil conversion equation
CrudConv = Equation(m, name="CrudConv", domain=[p,t], description="calculate ammount of expected ConvProd")
CrudConv[p,t] = ConvProd[p,t] == Sum(o, Crude_Conv_ratio[o,p] * Oil2Buy[o,t]) + Tot2Save[p,t-1]

# Crud converter Max
CrudConv_limit = Equation(m, name="CrudConv_limit", domain=[t], description="limit conversion")
CrudConv_limit[t] = Sum(p,ConvProd[p,t]) <= CrdConv_limit

# ConvProd to sell directly constraint
Direct_saleA = Equation(m, name="Direct_saleA", domain=[p,t], description="calculate ammount of prod A expected Conv2Tot")
Direct_saleA["A",t] = Conv2Tot["A",t] == ConvProd["A",t]

Direct_saleB = Equation(m, name="Direct_saleB", domain=[p,t], description="calculate ammount of prod B expected Conv2Tot") # Because you cant have quality and leq in the same equation :/
Direct_saleB["B",t] = Conv2Tot["B",t] <= ConvProd["B",t]

Direct_saleC = Equation(m, name="Direct_saleC", domain=[p,t], description="calculate ammount of prod C expected Conv2Tot")
Direct_saleC["C",t] = Conv2Tot["C",t] == 0

# ReConvProd conversion equation
ReConversion = Equation(m, name="ReConversion", domain=[p,t], description="calculate ammount of expected ConvProd")
ReConversion[p,t] = ReConv2Tot[p,t] == Sum(pp, Re_Conv_ratio[pp,p] * (ConvProd[pp,t]-Conv2Tot[pp,t])) # ConvProd[p,t]-Conv2Tot[p,t] = Conv2ReConv[p,t]

# Re converter limit
ReConc_limit = Equation(m, name="ReConc_limit", domain=[t], description="limit conversion")
ReConc_limit[t] = Sum(p,ReConv2Tot[p,t]) <= ReConv_limit

# Save parity equation
Save_parity = Equation(m, name="Save_parity", domain=[p,t], description="calculate ammount to be saved each week")
Save_parity[p,t] = Tot2Save[p,t] <= Conv2Tot[p,t] + ReConv2Tot[p,t]  # Conv2Tot[p,t] + ReConv2Tot[p,t] = Total products

# Sell limit constraint
Sell_limit_constraint = Equation(m, name="Sell_limit_constraint", domain=[p,t], description="limit sales")
Sell_limit_constraint[p,t] = Sell_limit[p,t] >= Conv2Tot[p,t] + ReConv2Tot[p,t] - Tot2Save[p,t] # Conv2Tot[p,t] + ReConv2Tot[p,t] - Tot2Save[p,t] Total products to be sold each day


# Objective function

In [820]:
obj = (Sum((p, t), Sell_Price[p] * (Conv2Tot[p,t] + ReConv2Tot[p,t] - Tot2Save[p,t]))
    - Sum((o, t), Purchase_Price[o] * Oil2Buy[o,t])
    - Sum((p, t), CrdConv_cost * ConvProd[p,t])
    - Sum((p, t), ReConv_cost * ReConv2Tot[p,t])
    - Sum((p, t), Storage_cost * Tot2Save[p,t])
)

# Solver

In [821]:
flow = Model(m, name="flow", equations=m.getEquations(), objective=obj, problem="LP", sense=Sense.MAX)
flow.solve(solver="CPLEX", options=Options(equation_listing_limit=200, variable_listing_limit=200))

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,118222.88961039,52,43,LP,CPLEX,0.001


In [822]:
Oil2Buy.records

,crude_oil,time,level,marginal,lower,upper,scale
0,co1,1,300.000000,97.402597,0.0,300.0,1.0
1,co1,2,46.875000,0.000000,0.0,300.0,1.0
2,co1,3,0.000000,-610.000000,0.0,300.0,1.0
3,co2,1,75.324675,0.000000,0.0,300.0,1.0
4,co2,2,0.000000,-117.187500,0.0,300.0,1.0
5,co2,3,0.000000,-714.000000,0.0,300.0,1.0


In [823]:
ConvProd.records

,products,time,level,marginal,lower,upper,scale
0,A,1,202.727273,0.0,0.0,inf,1.0
1,A,2,23.437500,0.0,0.0,inf,1.0
2,A,3,0.000000,0.0,0.0,inf,1.0
3,B,1,105.064935,0.0,0.0,inf,1.0
4,B,2,14.062500,0.0,0.0,inf,1.0
5,B,3,0.000000,0.0,0.0,inf,1.0
6,C,1,67.532468,0.0,0.0,inf,1.0
7,C,2,9.375000,0.0,0.0,inf,1.0
8,C,3,0.000000,0.0,0.0,inf,1.0


In [824]:
Conv2Tot.records

,products,time,level,marginal,lower,upper,scale
0,A,1,202.727273,0.000000,0.0,inf,1.0
1,A,2,23.437500,0.000000,0.0,inf,1.0
2,A,3,0.000000,0.000000,0.0,inf,1.0
3,B,1,105.064935,0.000000,0.0,inf,1.0
4,B,2,14.062500,0.000000,0.0,inf,1.0
5,B,3,0.000000,-20.000000,0.0,inf,1.0
6,C,1,0.000000,-226.534091,0.0,inf,1.0
7,C,2,0.000000,-524.375000,0.0,inf,1.0
8,C,3,0.000000,-0.000000,0.0,inf,1.0


In [825]:
ReConv2Tot.records

,products,time,level,marginal,lower,upper,scale
0,A,1,47.272727,0.0,0.0,inf,1.0
1,A,2,6.562500,0.0,0.0,inf,1.0
2,A,3,0.000000,-100.0,0.0,inf,1.0
3,B,1,20.259740,0.0,0.0,inf,1.0
4,B,2,2.812500,0.0,0.0,inf,1.0
5,B,3,0.000000,-100.0,0.0,inf,1.0
6,C,1,0.000000,0.0,0.0,inf,1.0
7,C,2,0.000000,-80.0,0.0,inf,1.0
8,C,3,0.000000,-80.0,0.0,inf,1.0


In [826]:
Tot2Save.records

,products,time,level,marginal,lower,upper,scale
0,A,1,0.0,-272.191558,0.0,inf,1.0
1,A,2,0.0,-686.250000,0.0,inf,1.0
2,A,3,0.0,0.000000,0.0,inf,1.0
3,B,1,0.0,-120.000000,0.0,inf,1.0
4,B,2,0.0,-860.000000,0.0,inf,1.0
5,B,3,0.0,0.000000,0.0,inf,1.0
6,C,1,0.0,0.000000,0.0,inf,1.0
7,C,2,0.0,-120.000000,0.0,inf,1.0
8,C,3,0.0,-20.000000,0.0,inf,1.0


In [827]:
sum(Oil2Buy.records.level)

422.19967532467535

In [828]:
sum(ConvProd.records.level)

422.19967532467535

In [829]:
sum(Conv2Tot.records.level)+sum(ReConv2Tot.records.level)

422.19967532467535